## Телеграмм Бот 
для отправки сообщений на канал https://t.me/MTSProject_StackMarket



In [2]:
!pip install aiogram
!pip install nest-asyncio

     |████████████████████████████████| 182 kB 5.5 MB/s 
     |████████████████████████████████| 1.3 MB 33.4 MB/s 
     |████████████████████████████████| 142 kB 66.7 MB/s 
     |████████████████████████████████| 294 kB 57.7 MB/s 


In [1]:
import os
import glob
import numpy as np
from datetime import datetime, date

In [3]:
# Подключение Гугл-диска. Надо подключить эту папку: https://drive.google.com/drive/u/5/folders/1ByM63BCupxb182WIY_HY14-o_77_vdGU?usp=sharing, можно сделать у себя ссылку на неё и обращаться как обычно (по сути добавить ярлык на папку)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Важно чтобы примонтированный каталог имел навзание "MTS.Project"
path_project = "drive/MyDrive/MTS.Project/"
path_prod = path_project + "Prod"
os.chdir("/")
os.chdir("content/" + path_prod)
print(os.getcwd())

/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod


In [7]:
# Доп библиотека для асинхронного вызова функций в colab
import nest_asyncio
nest_asyncio.apply()

In [8]:
import asyncio
from aiogram import Bot, types
from aiogram.dispatcher import Dispatcher
from aiogram.utils import executor

from config import TOKEN, CHANNEL_NAME, CHANNEL_ID

bot = Bot(token=TOKEN, parse_mode=types.ParseMode.HTML)
dp = Dispatcher(bot)

async def send_message(text: str):
    await bot.send_message(CHANNEL_ID, text)

# asyncio.run(send_message("Hello world!"))
# asyncio.get_event_loop().run_until_complete(main())

In [9]:
from PredictRecommendation import PredictRecommendation, PredictRecommendationTrend

In [10]:
# Формируем прогноз 
# 100 раз пожаелел, что поулчаем от класса PredictRecommendation готовые строки, надо было сразу все переводить в множества set  или отдельную структуру делать (что гораздо лучше)
path_model = "models/"
rec = ""
rec_array = []
# dirlist = os.listdir(path_model)
# Dense
dirlist = glob.glob(path_model + f"*model_Dense*")
for file_model in dirlist:
	print(file_model)
	# recommend = PredictRecommendation(model_path = path_model + file_model, is_log = False)
	recommend = PredictRecommendation(model_path = file_model, is_log = True)
	recommend.load_DataSet()
	symbols_df_normalize = recommend.normalize_Dataset(path_normailzer = "Normalizer")
	x = recommend.create_x()
	# Для модели dense необходимо передавать значения за текущий и предыдущий день. Решение о покупке или продаже принимается по двум предикатам 
	# last_day_x = x[-1][None]
	last_day_x = x[-2:]
	predict = recommend.predict_y(last_day_x)
	if not recommend.errors.strip():
		rec_str = recommend.get_recomend()
		if rec_str.strip():
			rec_array.append(rec_str)
	else:
		print("Выявлены ошибки!")
		print(recommend.errors)

# LSTM + Conv1D
dirlist = glob.glob(path_model + f"*model_Conv1D*") + glob.glob(path_model + f"*model_LSTM*")
for file_model in dirlist:
	print(file_model)
	# recommend = PredictRecommendation(model_path = path_model + file_model, is_log = False)
	recommend = PredictRecommendation(model_path = file_model, is_log = False)
	recommend.load_DataSet()
	symbols_df_normalize = recommend.normalize_Dataset(path_normailzer = "Normalizer")
	x = recommend.create_x()
	# Для модели dense необходимо передавать значения за текущий и предыдущий день. Решение о покупке или продаже принимается по двум предикатам 
	# last_day_x = x[-1][None]
	# last_day_x = x[-2:]

	# Для модели LSTM и Conv1D передаем вектор из 10 дней, но для предсказания нужны предикты за текущий и предыдузий дни
	last_day_x = [x[-8:]]
	last_day_x = np.array(last_day_x)
	last_day_predict = recommend.predict_y(last_day_x)
	
	pre_last_day_x = [x[-9:-1]]
	pre_last_day_x = np.array(pre_last_day_x)
	pre_last_day_predict = recommend.predict_y(pre_last_day_x)

	recommend.predict = np.array([pre_last_day_predict[0], last_day_predict[0]])
	# recommend.predict = [pre_last_day_predict[0], last_day_predict[0]]
	
	if not recommend.errors.strip():
		rec_str = recommend.get_recomend()
		if rec_str.strip():
			rec_array.append(rec_str)
	else:
		print("Выявлены ошибки!")
		print(recommend.errors)

show_rec = True 
today = datetime.now().strftime("%d.%m.%Y")
recommendation_text = f"Рекомендация на <b>{today}</b>:\n\t\t *По стратегии \"Корреляция в отрасли\" (среднедневная доходность стратегии: 0.49%):\n\t\t\t\t"

if len(rec_array) == 0:
	show_rec = False
else:
	if not "".join(rec_array).strip():
		rec = " Рекомендаций нет"
	else:
		rec = '\n\t\t\t\t'.join(rec_array)
	recommendation_text+=rec

print("show_rec: ", show_rec)
print("recommendation_text: ", recommendation_text)


models/SIBN_CII_1_1_model_Dense_acc_6697.h5
models/SIBN_CII_1_1_model_Dense_acc_6697.h5
Dense
SIBN
/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod
['SIBN_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'ROSN_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'SNGSP_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'GAZP_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'LKOH_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'NVTK_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv']


/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod/PredictRecommendation.py:67: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  mydateparser = lambda x: pd.datetime.strptime(x, "%Y.%m.%d %H:%M").date()


Кол-во уникальных индексов общих для всех датасетов:  13
x_input =>  [[0 0 0 0 1 1]
 [1 1 1 1 0 1]]
predict =>  [0.2692064 0.5660682]
 type_model => Dense, symbol => SIBN, predict => [0.2692064 0.5660682]
models/LKOH_CII_1_1_model_Dense_acc_6606.h5
models/LKOH_CII_1_1_model_Dense_acc_6606.h5
Dense
LKOH
/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod
['SIBN_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'ROSN_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'SNGSP_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'GAZP_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'LKOH_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'NVTK_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv']
Кол-во уникальных индексов общих для всех датасетов:  13
x_input =>  [[0 0 0 0 1 1]
 [1 1 1 1 0 1]]
predict =>  [0.35034907 0.5141791 ]
 type_model => Dense, symbol => LKOH, predict => [0.35034907 0.5141791 ]
models/LKOH_CII_1_1_model_Conv1D_acc_6481.h5


/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod/PredictRecommendation.py:67: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  mydateparser = lambda x: pd.datetime.strptime(x, "%Y.%m.%d %H:%M").date()


/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod
['SIBN_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'ROSN_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'SNGSP_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'GAZP_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'LKOH_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'NVTK_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv']


...

 type_model => Conv1D, symbol => SIBN, predict => [0.99907994 0.9125633 ]
models/LKOH_CII_1_1_model_LSTM_acc_6944.h5
/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod
['SIBN_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'ROSN_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'SNGSP_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'GAZP_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'LKOH_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'NVTK_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv']


/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod/PredictRecommendation.py:67: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  mydateparser = lambda x: pd.datetime.strptime(x, "%Y.%m.%d %H:%M").date()


 type_model => LSTM, symbol => LKOH, predict => [0.5724014  0.19554806]
models/SIBN_CII_1_1_model_LSTM_acc_6019.h5
/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod
['SIBN_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'ROSN_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'SNGSP_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'GAZP_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'LKOH_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv', 'NVTK_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv']


/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod/PredictRecommendation.py:67: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  mydateparser = lambda x: pd.datetime.strptime(x, "%Y.%m.%d %H:%M").date()


 type_model => LSTM, symbol => SIBN, predict => [0.607054  0.5762683]
show_rec:  True
recommendation_text:  Рекомендация на <b>01.09.2021</b>:
		 *По стратегии "Корреляция в отрасли" (среднедневная доходность стратегии: 0.49%):
				<i> - Покупка SIBN (уверенность модели Conv1D: 91%)</i>
				<i> - Покупка SIBN (уверенность модели LSTM: 57%)</i>


In [11]:

path = "DataSet/"
today = datetime.now().strftime("%d_%m_%Y")
dataset_path = path + "GazNeft_D1/" + today + '/'
os.listdir(dataset_path)


['SIBN_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv',
 'ROSN_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv',
 'SNGSP_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv',
 'GAZP_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv',
 'LKOH_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv',
 'NVTK_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv']

In [12]:
path = "DataSet/GazNeft_D1/"
		# Опреденляем текущий день
today = datetime.now().strftime("%d_%m_%Y")
		# Получаем список файлов
print(path + "GazNeft_D1/" + today )
try:
	dataset_path = path + today + '/'
	dirlist = os.listdir(dataset_path)
 
except:
  None
dirlist

DataSet/GazNeft_D1/GazNeft_D1/01_09_2021


['SIBN_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv',
 'ROSN_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv',
 'SNGSP_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv',
 'GAZP_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv',
 'LKOH_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv',
 'NVTK_D1_HLOCV_MTS_Project_1_9_2021___10-28-43.csv']

In [13]:
path_project = "drive/MyDrive/MTS.Project/"
path_prod = path_project + "Prod"
os.chdir("/")
os.chdir("content/" + path_prod)
print(os.getcwd())

/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod


In [14]:
path_model_direction =  "trend_direction_models/"
path_model_strength =  "trend_strength_models/"	
dirlist_trend_strength = os.listdir(path_model_strength)
dirlist_trend_direction = os.listdir(path_model_direction)	
rec_trend = "" 

for i in range(len(dirlist_trend_strength)):
    file_model_strength = dirlist_trend_strength[i]
    current_symbol = dirlist_trend_strength[i].split('_')[-1]
    for k in range (len(dirlist_trend_direction)):
        if dirlist_trend_direction[k].split('_')[-1] == current_symbol:
          
          file_model_direction = dirlist_trend_direction[k]      
          #print (file_model_strength, file_model_direction)
          recommend = PredictRecommendationTrend(model_strength_path = path_model_strength + file_model_strength, model_direction_path=path_model_direction + file_model_direction, is_log = False)
          
      
        #   recommend.load_DataSet(path = os.getcwd()+"DataSet/GazNeft_D1/") 
          recommend.load_DataSet() 
          #print (recommend.load_DataSet())
          symbols_df_normalize = recommend.normalize_Dataset(path_normailzer="Trend_normalizer")
          #print(symbols_df_normalize)
          x = recommend.create_x()
          predict = recommend.predict(x)
          rec_trend += recommend.get_recomend()

if rec_trend.strip() == "":
	  rec_trend = "\n\tРекомендаций нет"


recommendation_text_trend =  f"\n\n\t\t * По стратегии \"Предсказание тренда\" (среднедневная доходность стратегии: 0.10%)\t\t\t\t"
recommendation_text_trend+=rec_trend

recommendation_text+=recommendation_text_trend #Изменил recommendation_text
print("recommendation_text: ", recommendation_text)

/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod/PredictRecommendation.py:193: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  mydateparser = lambda x: pd.datetime.strptime(x, "%Y.%m.%d %H:%M").date()
/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod/PredictRecommendation.py:193: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  mydateparser = lambda x: pd.datetime.strptime(x, "%Y.%m.%d %H:%M").date()
/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod/PredictRecommendation.py:193: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  mydateparser = lambda x: pd.datetime.strptime(x, "%

recommendation_text:  Рекомендация на <b>01.09.2021</b>:
		 *По стратегии "Корреляция в отрасли" (среднедневная доходность стратегии: 0.49%):
				<i> - Покупка SIBN (уверенность модели Conv1D: 91%)</i>
				<i> - Покупка SIBN (уверенность модели LSTM: 57%)</i>

		 * По стратегии "Предсказание тренда" (среднедневная доходность стратегии: 0.10%)				
	SNGSP: Покупать (Предположительное направление тренда – вверх; Вероятность дальнейшего продолжения тренда – большая)
	LKOH: Продавать (Предположительное направление тренда – вниз; Вероятность дальнейшего продолжения тренда – большая)
	NVTK: Покупать (Предположительное направление тренда – вверх; Вероятность дальнейшего продолжения тренда – большая)
	GAZP: Продавать (Предположительное направление тренда – вниз; Вероятность дальнейшего продолжения тренда – большая)


/content/drive/.shortcut-targets-by-id/1ByM63BCupxb182WIY_HY14-o_77_vdGU/MTS.Project/Prod/PredictRecommendation.py:193: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  mydateparser = lambda x: pd.datetime.strptime(x, "%Y.%m.%d %H:%M").date()


In [15]:
# Если не было ошибок, то отправляем сообщение в телеграм канал 
if show_rec:
  asyncio.run(send_message(recommendation_text))